<a href="https://colab.research.google.com/github/nxxk23/AI-Engineer/blob/main/deepcut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install protobuf==3.19.5

In [ ]:
!pip install transformers
!pip install qdrant-client>=1.1.1
!pip install pythainlp

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F
from torch import Tensor
from pythainlp.tokenize import word_tokenize
from qdrant_client import models

In [7]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/AIEngineer/data.csv")
df = df.drop(columns=['Unnamed: 0'])
df['sentence'] = df['clean_subject'] + ' ' + df['clean_description']
df = df[df['ticket_type'] != 'Problem'] #Drop Class Problem
df.dropna(subset=['sentence'], inplace=True)
df['sentence'].loc[0]

'ma     น      น   cid  inextbaac   สาขา หน่วยบุรีรัมย์ เรื่อง ขอกำหนดการปรับปรุงการให้บริการ บริษัท ไอเน็กซ์ บรอดแบนด์ จำกัด ขอจัดส่งจดหมายปรับปรุงการบริการวงจร จำนวน  วงจร วัตถุประ์การดำเนินการ   ปรับปรุงบริการโครงข่าย เนื่องจากจะดำเนินการ replace อุปกรณ์ภายใน node พื้นที่ดำเนินการ   node บุรีรัมย์ site interlink วัน เริ่มดำเนินการ       น  วัน สิ้นสุดดำเนินการ       น  ระยะกระทบ   กระทบบริการไม่เกิน   ชั่วโมง รายละเอียดทั้งหมดตาม เอกสารแนบมา จึงมาโปรดทราบความจำเป็นดังกล่าว และบริษัทฯ ขออภัยในความไม่สะดวกในครั้งนี้ หากท่านมีข้อสงสัยหรือต้องการข้อมูลเพิ่มเติม โปรดติดต่อตามหมายเลขด้านล่าง หรือ reply mail กลับมาที่'

In [8]:
doc = df[['sentence', 'ticket_type']].sample(n=1000)
sen_list = doc.to_dict(orient='records')
sen_list[0]

{'sentence': 'softway    express cloud   สรใหม่ wt เจ้าหน้าที่ไอเน็ท การใช้บริการ express on cloud การใช้บริการ      สถานะการใช้งาน   สรใหม่ และ ขอ exe สรใหม่ company   บ ที เอส อินเตอร์ฟู๊ด เทรดดิ้ง username   wt  จำนวน user    users host    วันเริ่มสุดสัญญา      วันสิ้นสุดสัญญา      ระยะสัญญา    เดือน หมายเหตุ',
 'ticket_type': 'Request'}

In [9]:
doc['ticket_type'].value_counts(normalize = True)

ticket_type
Request     0.59
Incident    0.41
Name: proportion, dtype: float64

In [14]:
!pip install --upgrade tensorflow pythainlp

In [15]:
def average_pool(last_hidden_states: Tensor, attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

# Using PyThaiNLP to tokenize the input texts
input_texts = [' '.join(word_tokenize(doc['sentence'], engine="deepcut")) for doc in sen_list]

# Load transformers model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large-instruct')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large-instruct')

# Tokenize the input texts
batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')

# Get model outputs
outputs = model(**batch_dict)
embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

# Normalize embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)

# Connect to Qdrant
qdrant = QdrantClient(":memory:")

# Create a collection in Qdrant
qdrant.recreate_collection(
    collection_name="test",
    vectors_config=models.VectorParams(
        size=embeddings.size(-1),  # Use the size of the embeddings
        distance=models.Distance.COSINE
    )
)

# Prepare points to insert
points = [
    PointStruct(
        id=i,
        vector=embeddings[i].tolist(),
        payload={"document": sen_list[i]}
    )
    for i in range(len(sen_list))
]

# Insert points into the collection
qdrant.upsert(
    collection_name="test",
    points=points
)


OSError: /usr/local/lib/python3.10/dist-packages/tensorflow/python/platform/../../core/platform/_cpu_feature_guard.so: cannot open shared object file: No such file or directory

In [ ]:
# Example query
query = 'สาขา หน่วยบุรีรัมย์ เรื่อง ขอกำหนดการปรับปรุงการให้บริการ บริษัท ไอเน็กซ์ บรอดแบนด์ จำกัด ขอจัดส่งจดหมายปรับปรุงการบริการวงจร จำนวน  วงจร วัตถุประการดำเนินการ'
query_text = ' '.join(word_tokenize(query, engine="deepcut"))
query_dict = tokenizer(query_text, max_length=512, padding=True, truncation=True, return_tensors='pt')

query_outputs = model(**query_dict)
query_embedding = average_pool(query_outputs.last_hidden_state, query_dict['attention_mask'])

query_embedding = F.normalize(query_embedding, p=2, dim=1)

# Search in Qdrant
search_result = qdrant.search(
    collection_name="my_collection",
    query_vector=query_embedding[0].tolist(),
    limit=3  # Number of nearest neighbors to return
)

# Output search results
for i, hit in enumerate(search_result):
    print(f"Result {i+1}:")
    print("Score:", hit.score)
    print("Payload:", hit.payload)
    print(f"--------------------")
